In [ ]:
# decompress provided save folder from the qe calculation
!tar -xzvf temp.tar.gz

In [ ]:
import pw2py as pw
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack

### qesave is a module

In [ ]:
# check out the available functions
[d for d in dir(pw.qesave) if not d.startswith('_')]

### Reading charge density

In [ ]:
help(pw.qesave.read_charge_density)

In [ ]:
savefolder = 'temp/BN.save'
rho = pw.qesave.read_charge_density(savefolder)
print(rho)
print(rho.shape) # same as fft grid

In [ ]:
# average charge density in the xy direction
rho_z = np.average(rho, axis=(0, 1))
print(rho_z)
print(rho_z.shape)

In [ ]:
# plot rho_z
inp = pw.qeinp.from_file('scf.in')
par_z = inp.par[2, 2]

zvals = np.linspace(0, par_z, num=(rho_z.size+1))[:-1]

plt.plot(zvals, rho_z)

plt.ylabel(r'$\rho$ [a.u.]')
plt.xlabel(r'$z$ [a.u.]')

plt.tight_layout()
plt.savefig('rho.png')
plt.show()

### Reading wavefunctions

In [ ]:
help(pw.qesave.read_wavefunction)

In [ ]:
evc = pw.qesave.read_wavefunction(savefolder)
print(evc)
print("(kpt, spin, band, gvec_index) -> ", evc.shape)
print('Note! While the calculation in this folder was actually run with a 3x3 grid only the kp K00001 (i.e. gamma) is kept in the tutorial to save storage space')

In [ ]:
# this calculation is a simple 4x4 h-BN lattice with a carbon defect
# let's take a closer look at band 65, spin up, and at the gamma point
evc65 = evc[0, 0, 65]
print(evc65)
print(evc65.shape)
print(np.linalg.norm(evc65)) # we used norm-conserving pseudopotentials so the norm should be 1

In [ ]:
# as you can see the wavefunction is complex, but is actually stored as a 1d array
# also this function is defined in G space (whereas rho above was defined in R space)
# read the g vectors using pw2py
gk = pw.qesave.read_gkvectors(savefolder)[0]
print(gk)
print(gk.shape)

In [ ]:
# what if we want the wavefunction defined in real space e.g. psi(R)?
# well will need to reshape evc65 according to gk above and then preform a fourier transform on psi(G) -> psi(R)
# thankfully pw2py has you covered with a semi-decent implementation
help(pw.functions.reshape_wfc3D)

In [ ]:
# use pw.functions.reshpae_wfc3D to reshape evc65 to an explicitly 3-dimensional array we'll call evc3D
evc3D = pw.functions.reshape_wfc3D(evc65, gk, scaling=2)
print(evc3D)
print(evc3D.shape)

In [ ]:
# now we can use scipy to fourier transform our wfc from G space to R space
evcR = scipy.fftpack.fftn(evc3D, overwrite_x=False) / (evc3D.size)**0.5
print(evcR)
print(evcR.shape)
print(np.linalg.norm(evcR)) # the norm should still be 1!

In [ ]:
# now that we have the real space wavefunction let's visualize it similarly to how we did above for rho
evcR_x = np.average(np.abs(evcR), axis=(1, 2))

par_x = inp.par[0, 0]
xvals = np.linspace(0, par_x, num=(evcR_x.size+1))[:-1]

plt.plot(xvals, evcR_x)

plt.ylabel(r'$\psi(x)$')
plt.xlabel(r'$x$ [a.u.]')

plt.tight_layout()
plt.savefig('evc.png')
plt.show()

In [ ]:
# some things are already implemented straightforwardly
help(pw.functions.plot_wfc_xsf)
pw.functions.plot_wfc_xsf('evc_65_up.xsf', inp, evc65, gk, lsign=True)

In [ ]:
# another example
help(pw.functions.plot_wfc_averaged)
pw.functions.plot_wfc_averaged('evc_65_z.txt', evc65, gk)